<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.01/notebook/01_1_IR_Assignment_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrival Course

## Assigment 1

### Step 1: Data Cleaning

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# cleaning the data
!rm data.xlsx

rm: cannot remove 'data.xlsx': No such file or directory


In [ ]:
# downloding the data
import requests
import os
import pandas as pd

url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/data.xlsx?raw=true"
output_filename = "data.xlsx"

response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as file:
        file.write(response.content)

print(f"The file was downloaded, and it is in {output_filename}.")

The file was downloaded, and it is in data.xlsx.


In [ ]:
data_aj = pd.read_excel(output_filename, sheet_name="A-J", engine="openpyxl")
data_bbc = pd.read_excel(output_filename, sheet_name="BBC", engine="openpyxl")
data_jp = pd.read_excel(output_filename, sheet_name="J-P", engine="openpyxl")
data_nyt = pd.read_excel(output_filename, sheet_name="NY-T", engine="openpyxl")

## Step 1.1 - Preaparing the data

We need to prepare the data each source at a time due to difference in the naming convenstion.

In [ ]:
# processing the data for aj

col_names_aj = ['title', 'sub_title', 'Body Text']
# we will add all the text from the 3 column above (is nan replace by "")
# we will add a column 'id' that will be aj_<i> where i is the index of the row

data_aj = data_aj[col_names_aj]
data_aj = data_aj.fillna("")

df_aj = pd.DataFrame()
df_aj["id"] = range(1, len(data_aj) + 1)
df_aj["id"] = "aj_" + df_aj["id"].astype(str)
df_aj["document"] = data_aj["title"] + " " + data_aj["sub_title"] + " " + data_aj["Body Text"]

print(df_aj.head())

     id                                           document
0  aj_1  pope renews call for gaza ceasefire, release o...
1  aj_2  biden is still the best us president israel co...
2  aj_3  israeli air strikes continue across gaza as tr...
3  aj_4  police remove pro-palestinian students from pa...
4  aj_5  mass graves found at southern gaza hospital ra...


In [ ]:
# processing the data for bbc
col_names_bbc = ['title', "Body Text"]

data_bbc = data_bbc[col_names_bbc]
data_bbc = data_bbc.fillna("")


df_bbc = pd.DataFrame()
df_bbc["id"] = range(1, len(data_bbc) + 1)
df_bbc["id"] = "bbc_" + df_bbc['id'].astype(str)
df_bbc["document"] = data_bbc["title"] + " " + data_bbc["Body Text"]

print(df_bbc.head())

      id                                           document
0  bbc_1  tanks push further into khan younis as israel ...
1  bbc_2  kuehne+nagel in milton keynes 'vandalised by p...
2  bbc_3  eisenkot: key israeli war leader challenges ne...
3  bbc_4  how gunfire and fear engulfed gaza hospital be...
4  bbc_5  who fears for remaining patients at gaza's nas...


In [ ]:
# processing the data for jp
col_names_jp = ['title', "Body"]

data_jp = data_jp[col_names_jp]
data_jp = data_jp.fillna("")

df_jp = pd.DataFrame()
df_jp["id"] = range(1, len(data_jp) + 1)
df_jp["id"] = "jp_" + df_jp['id'].astype(str)
df_jp["document"] = data_jp["title"] + " " + data_jp["Body"]

print(df_jp.head())

     id                                           document
0  jp_1  'loved but not lost': piano ensemble presents ...
1  jp_2  is jerusalem’s mughrabi bridge in news to spar...
2  jp_3  israeli ‘extra-judicial executions' slammed by...
3  jp_4  masa israel journey to mark remembrance day wi...
4  jp_5  protesters gather across israel, calling for h...


In [ ]:
# processing the data for nty
col_names_nyt = ['title', 'Body Text']

data_nyt = data_nyt[col_names_nyt]
data_nyt = data_nyt.fillna("")

df_nyt = pd.DataFrame()
df_nyt["id"] = range(1, len(data_nyt) + 1)
df_nyt["id"] = "nyt_" + df_nyt['id'].astype(str)
df_nyt["document"] = data_nyt["title"] + " " + data_nyt["Body Text"]

print(df_nyt.head())

      id                                           document
0  nyt_1  hezbollah launches rocket barrage after israel...
1  nyt_2  gaza offensive to last at least to year’s end,...
2  nyt_3  trapped and starving, 2 families in gaza try t...
3  nyt_4  u.n. says israel may be restricting gaza aid a...
4  nyt_5  help! we’re booked on a nile cruise but worry ...


Now that all the data is in the same format, at data frame of 2 x length of data with 2 column 'id', 'document' and non are empty and contain the concatunation of the title + sub_title + body (if exists)

In [ ]:
print(f'Number of document from AJ:\t {len(df_aj)}')
print(f'Number of document from BBC:\t {len(df_bbc)}')
print(f'Number of document from JP:\t {len(df_jp)}')
print(f'Number of document from NYT:\t {len(df_nyt)}')

Number of document from AJ:	 599
Number of document from BBC:	 549
Number of document from JP:	 599
Number of document from NYT:	 599


Now we have 4 df with the column of text and title for each one of the 4 source and each data frame has about 600 articles.

Now we will perform the data cleaning in two steps:

1. cleaning the word from all the punctuation marks.
  - for example "How are you?" -> "How are you ?" because "you" != "you?"
2. cleaning the documents with lemmatisation.
 - "cleaning" -> "clean"

 Outpute at the end:

 1. 4 cleaned document sets from the ponctuation (not deleting them).
 2. 4 cleaned document set by that include only the lemma (not word but their root).

## Step 1.2 - Word

In [ ]:
# before we perform any cleaning of any text we will do the following:
# we will replace all the different kind of '` etc to a standard '
# we will replace all the different kind of " to the standar "

import re

def clean_text(text):
  # Normalize all types of single and double quotation marks to standard forms
  text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
  text = re.sub(r"[“”]", '"', text)   # Convert all double quote variations to "

  return text

In [ ]:
# now we will clean our data from the hazard

df_aj['document'] = df_aj['document'].apply(clean_text)
df_bbc['document'] = df_bbc['document'].apply(clean_text)
df_jp['document'] = df_jp['document'].apply(clean_text)
df_nyt['document'] = df_nyt['document'].apply(clean_text)

In [ ]:
# performing the cleaning number 1 separeting the words from the punctuation
# e.g. 'What is your name?' -> 'What is your name ?'
# while "doesn't" -> "doesn't"
import re

def clean_text(text):
  # Tokenize with regex to handle punctuation outside of words and contractions
  tokens = re.findall(r"\b\w+(?:'\w+)?\b|[^\w\s]", text, re.UNICODE)

  return ' '.join(tokens)

def clean_df_punctuation(df):
  df_copy = df.copy()
  df_copy["document"] = df_copy["document"].astype(str).apply(clean_text)
  return df_copy

In [ ]:
# test the cleaning with a df
# we can perform more tests as we find extreme cases.

text = "How are you? I'm well thank you. doesn't."
clean_text(text)

"How are you ? I'm well thank you . doesn't ."

In [ ]:
# clean all the data
df_aj_word = clean_df_punctuation(df_aj)
df_bbc_word = clean_df_punctuation(df_bbc)
df_jp_word = clean_df_punctuation(df_jp)
df_nyt_word = clean_df_punctuation(df_nyt)

In [ ]:
# test the first few lines

print(df_aj_word['document'].head())

0    pope renews call for gaza ceasefire , release ...
1    biden is still the best us president israel co...
2    israeli air strikes continue across gaza as tr...
3    police remove pro - palestinian students from ...
4    mass graves found at southern gaza hospital ra...
Name: document, dtype: object


## Step 1.3 - Lemmatization

Now that we have clean the data in the first type we will do the lemmatisation.

In [ ]:
# installing the needed packeges
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 81.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("stopwords")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import spacy

# in this part we will:
# 1. remove from the data all the 's at the end of word and such
# 2. remove all the stop words
# 2. lemmatize the for

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words("english"))


def clean_text_lemma(text):
  # replace all the I'm to I am etc
  text = re.sub(r"I'm", "I am", text)
  text = re.sub(r"you're", "you are", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"we're", "we are", text)
  text = re.sub(r"they're", "they are", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"how's", "how is", text)
  text = re.sub(r"i'll", "I will", text)

  # remove from the text all the punctatution
  text = re.sub(r'[^\w\s]', '', text)

  # tokenize the text
  tokens = word_tokenize(text)

  # remove all the numbers and dates etc
  tokens = [word for word in tokens if not any(char.isdigit() for char in word)]

  # remove the stopwords
  tokens = [word for word in tokens if not word.lower in stop_words]

  doc = nlp(' '.join(tokens))
  lemmatized_text = ' '.join(token.lemma_ for token in doc)
  return lemmatized_text

def clean_df_lemma(df):
  df_copy = df.copy()
  df_copy["document"] = df_copy["document"].astype(str).apply(clean_text_lemma)
  return df_copy

In [ ]:
# test the clean_text_lemma function

text = "How are you? I'm well thank you. doesn't. cleaning busiest 2024 Mon 20nd of June i'll "
print(clean_text_lemma(text))

how be you I be well thank you do not clean busy Mon of June I will


Ok now that we have a lemmatization function that works even on the extrem cases lets clean the text

In [ ]:
# clean the data
df_aj_lemma = clean_df_lemma(df_aj)
df_bbc_lemma = clean_df_lemma(df_bbc)
df_jp_lemma = clean_df_lemma(df_jp)
df_nyt_lemma = clean_df_lemma(df_nyt)

In [ ]:
for i in range(5):
  print(f"{i + 1}:\n{df_aj_lemma['document'][i]}")
  print(f"{df_aj['document'][i]}\n\n")

1:
pope renews call for gaza ceasefire release of captive in easter address Pope Francis say his thought go to those face war especially child who have forget how to smile pope francis have renew call for an immediate ceasefire in gaza and the release of all israeli captive in a peacefocuse address mark easter sunday the most important day on the christian calendar
pope renews call for gaza ceasefire, release of captives in easter address Pope Francis says his thoughts go to those facing wars, especially children who have 'forgotten how to smile'. pope francis has renewed calls for an immediate ceasefire in gaza and the release of all israeli captives in a peace-focused address marking easter sunday, the most important day on the christian calendar.


2:
biden be still the good us president israel could wish for the meaningless ceasefire resolution his administration allow the UN Security Council to pass should not fool anyone united states president ronald reagan order to israeli prim

In [ ]:
# store the all the cleaned data in files for further usage
# the name of the file should be <source>_word.csv with header

df_aj_word.to_csv("A_J_word.csv", index=False)
df_bbc_word.to_csv("BBC_word.csv", index=False)
df_jp_word.to_csv("J_P_word.csv", index=False)
df_nyt_word.to_csv("NYT_word.csv", index=False)

In [ ]:
df_aj_lemma.to_csv("A_J_lemma.csv", index=False)
df_bbc_lemma.to_csv("BBC_lemma.csv", index=False)
df_jp_lemma.to_csv("J_P_lemma.csv", index=False)
df_nyt_lemma.to_csv("NYT_lemma.csv", index=False)